In [ ]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
n_pages = 48
n_votes = str(20)
pages = [str(25*i+1) for i in range(n_pages)]
artist = []
album_name = []
year = []
critic_score = []
user_score = []
gap_score = []

In [ ]:
for p in range(1,n_pages+1,1):
    print(p)
    url = "https://www.albumoftheyear.org/ratings/6-highest-rated/2010s/" + str(p) + "?r=" + n_votes
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    album_container = html_soup.find_all('div', class_ = 'albumListRow')
    
    for album in album_container:
        sub_url = album.a.get('href')
        sub_url = "https://www.albumoftheyear.org"+sub_url
        
        response_sub_url = get(sub_url)
        soup_sub_url = BeautifulSoup(response_sub_url.text, 'html.parser')
        
        #---artist
        container_sub_url = soup_sub_url.find_all("div",class_="artist")
        artist.append(container_sub_url[0].a.text)
        
        #---album
        container_sub_url = soup_sub_url.find_all("div",class_="albumTitle")
        album_name.append(container_sub_url[0].span.text)
        
        #---critic
        container_sub_url = soup_sub_url.find_all("div",class_="albumCriticScore")
        critic_score.append(int(container_sub_url[0].a.text))
        
        #---user
        container_sub_url = soup_sub_url.find_all("div",class_="albumUserScore")
        user_score.append(int(container_sub_url[0].a.text))
        

In [ ]:
albums_ratings = pd.DataFrame({"Artist":artist})
albums_ratings.insert(1,"Album",album_name,True)
albums_ratings.insert(2,"User Score",user_score,True)
albums_ratings.insert(3,"Critic Score",critic_score,True)
albums_ratings.insert(4,"Gap Score",[a_i - b_i for a_i, b_i in zip(critic_score, user_score)],True)

albums_ratings

In [ ]:
albums_ratings.to_csv('aoty_2010s_best.csv',index=False) 

In [ ]:
albums_ratings.sort_values(by=['Gap Score'], inplace=True,ascending=True)
albums_ratings

In [ ]:
albums_ratings[['Artist','Album','Gap Score']][:20]
